# The PIF structure
The Physical Information File (PIF) is a __general__, __flexible__, and __hierachical__ schema for representing infomation about physical devices and materials. This enables the PIF to store a wide range of information on many kinds of physical systems, but requires more careful thought on where to store information within the schema.


Full documentation is available at [citrine.io/pif](citrine.io/pif)

* The PIF focuses on information specific to physical systems, but is sufficiently __general__ to handle a wide range of systems.
* The PIF does not impose a rigid schema, and instead is __flexible__ in exactly where data is stored
* The PIF has a __hierarchical__ structure can store information on multiple levels

## System

A system is the primary building block in a PIF record. Systems contain three general categories of information relevant to physical systems:

* Identifiers - What is this?

* Preparation - How was this made?

* Properties - How does this perform and what are its characteristics?

In [1]:
from pypif import pif
from pypif.obj import *

my_pif = ChemicalSystem()

print( pif.dumps(my_pif, indent=4))

{
    "category": "system.chemical"
}


## Identifiers
What is this?
* Formula, composition, name


In [2]:
my_pif.chemical_formula = "Li0.0024Ni0.9976O"

print( pif.dumps(my_pif, indent=4))

{
    "category": "system.chemical",
    "chemicalFormula": "Li0.0024Ni0.9976O"
}


## Preparation
How was this made?
* Processing information

In [3]:
route = ProcessStep(name="Route")
route.details = [Value(name="Solid state reaction")]

my_pif.processing = [route]

print( pif.dumps(my_pif, indent=4))

{
    "category": "system.chemical",
    "chemicalFormula": "Li0.0024Ni0.9976O",
    "processing": [
        {
            "name": "Route",
            "details": [
                {
                    "name": "Solid state reaction"
                }
            ]
        }
    ]
}


## Properties
How does this perform and what are its characteristics?
* Properties, measurement conditions


In [4]:
crystallinity = Property(name="Crystallinity", scalars="Polycrystalline")
resistivity = Property(name="Resistivity", units="\Ohm*cm", scalars=[28.8677, 0.2629, 0.0466])
resistivity.conditions = [Value(name="Temperature", units="K", scalars=[400, 700, 1000])]

power_factor = Property(name="Power factor", units="W/mK", scalars=[1.21E-4, 1.66E-2, 1.48E-1])
power_factor.conditions = [Value(name="Temperature", units="K", scalars=[400, 700, 1000])]

my_pif.properties = [crystallinity, resistivity, power_factor]

print( pif.dumps(my_pif, indent=4))

{
    "properties": [
        {
            "name": "Crystallinity",
            "scalars": "Polycrystalline"
        },
        {
            "name": "Resistivity",
            "scalars": [
                28.8677,
                0.2629,
                0.0466
            ],
            "units": "\\Ohm*cm",
            "conditions": [
                {
                    "name": "Temperature",
                    "scalars": [
                        400,
                        700,
                        1000
                    ],
                    "units": "K"
                }
            ]
        },
        {
            "name": "Power factor",
            "scalars": [
                0.000121,
                0.0166,
                0.148
            ],
            "units": "W/mK",
            "conditions": [
                {
                    "name": "Temperature",
                    "scalars": [
                        400,
                        700,
              

## Reference information


In [5]:
my_pif.references = [Reference(doi="10.1143/JJAP.38.L1336")]

print( pif.dumps(my_pif, indent=4))

{
    "references": [
        {
            "doi": "10.1143/JJAP.38.L1336"
        }
    ],
    "properties": [
        {
            "name": "Crystallinity",
            "scalars": "Polycrystalline"
        },
        {
            "name": "Resistivity",
            "scalars": [
                28.8677,
                0.2629,
                0.0466
            ],
            "units": "\\Ohm*cm",
            "conditions": [
                {
                    "name": "Temperature",
                    "scalars": [
                        400,
                        700,
                        1000
                    ],
                    "units": "K"
                }
            ]
        },
        {
            "name": "Power factor",
            "scalars": [
                0.000121,
                0.0166,
                0.148
            ],
            "units": "W/mK",
            "conditions": [
                {
                    "name": "Temperature",
               

## Hierarchical
* Subsystems

In [6]:
thermoelectric_module = System(name="Thermoelectric module")
thermoelectric_module.subsystems = [my_pif]

print( pif.dumps(thermoelectric_module, indent=4))

{
    "name": "Thermoelectric module",
    "category": "system",
    "subsystems": [
        {
            "references": [
                {
                    "doi": "10.1143/JJAP.38.L1336"
                }
            ],
            "properties": [
                {
                    "name": "Crystallinity",
                    "scalars": "Polycrystalline"
                },
                {
                    "name": "Resistivity",
                    "scalars": [
                        28.8677,
                        0.2629,
                        0.0466
                    ],
                    "units": "\\Ohm*cm",
                    "conditions": [
                        {
                            "name": "Temperature",
                            "scalars": [
                                400,
                                700,
                                1000
                            ],
                            "units": "K"
                        }

## Citrination metadata
Citrination identifiers for querying and navigation
* UID, Tags

In [7]:
my_pif.uid = "my-pif"
my_pif.tags = ["my_second_upload"]

print( pif.dumps(my_pif, indent=4))

{
    "tags": [
        "my_second_upload"
    ],
    "references": [
        {
            "doi": "10.1143/JJAP.38.L1336"
        }
    ],
    "uid": "my-pif",
    "properties": [
        {
            "name": "Crystallinity",
            "scalars": "Polycrystalline"
        },
        {
            "name": "Resistivity",
            "scalars": [
                28.8677,
                0.2629,
                0.0466
            ],
            "units": "\\Ohm*cm",
            "conditions": [
                {
                    "name": "Temperature",
                    "scalars": [
                        400,
                        700,
                        1000
                    ],
                    "units": "K"
                }
            ]
        },
        {
            "name": "Power factor",
            "scalars": [
                0.000121,
                0.0166,
                0.148
            ],
            "units": "W/mK",
            "conditions": [
       

### creating a dataset

If you have been completing the API Example tutorials in the order in which they're presented in the [README](README.md), feel free to use the dataset id you created previously. Otherwise, the cell below will use the `citrination_client` to create a new dataset and corresponding `dataset_id`.

In [29]:
#comment this cell if you have an ID from a dataset you created via the website

import random
import string
import json

from citrination_client import CitrinationClient
from os import environ

client = CitrinationClient(environ['CITRINATION_API_KEY'], 'https://citrination.com')


#the following lines create a dataset to use throughout the tutorial notebooks
random_string = ''.join([random.choice(string.ascii_uppercase + string.digits) for i in range(5)])
dataset_name = "Tutorial dataset " + random_string

dataset = client.data.create_dataset(name=dataset_name, description="Dataset for tutorial", public=False)

#your dataset_id will be stored here
dataset_id = dataset._id
print(dataset_id)

160722


In [34]:

filename = "example_data/pif.json"
with open(filename, 'w') as fp:
    pif.dump(my_pif, fp)
result = client.data.upload(dataset_id, filename)
result.__dict__

{'_failures': [], '_successes': [{'path': 'example_data/pif.json'}]}

Once uploaded, a PIF can be viewed at the url http://citrination.com/pifs/{dataset_id}/{version_number}/{uid}